### Custom IAM Policy for Role, Policy Management and Elastic Container Registry

 Steps:
- Log in to the AWS Management Console using your Admin or Root User credentials by visiting the [AWS Console login page](https://aws.amazon.com/console/).
- Attach a Managed Policy IAMFullAccess.
- Once you’ve attached the policy with sufficient permissions, switch back to your user and run the initial code to create roles, attach policies and create custom containers.

In [1]:
import os
import boto3
from RoleManager import RoleManager

%load_ext dotenv
%dotenv

Python-dotenv could not parse statement starting at line 15


In [2]:
from program.code.config.AWSClientManager import AWSClientManager

account_id = os.environ["ACCOUNT_ID"]
domain_id = os.environ["DOMAIN_ID"]
region = os.environ["AWS_REGION"]
bucket = os.environ["BUCKET"]
user_name = os.environ["USER_NAME"]
role = os.environ["ROLE"]
role_name = os.environ["ROLE_NAME"]
policy_name = os.environ["POLICY_NAME"]

role_service = RoleManager(account_id, user_name, role_name, policy_name, region)
role_service.create_role_and_policy()

Successfully updated trust relationship for role: AmazonSageMaker-ExecutionRole-20240309T101533


Policy AmazonSageMaker-ExecutionPolicy-20240309T101533 already exists.


In [3]:
aws_client_manager = AWSClientManager(account_id, role_name, region)
iam_client = aws_client_manager.get_client('iam')

In [4]:
from EcrManager import EcrManager

processor_image_name = 'sagemaker-processing-container'
train_image_name = 'xgb-clf-training-container'

ecr_client = aws_client_manager.get_client('ecr')
ecr_manager = EcrManager(ecr_client)

ecr_manager.create_repository(processor_image_name)
ecr_manager.create_repository(train_image_name)

{'repositoryArn': 'arn:aws:ecr:eu-north-1:284415450706:repository/xgb-clf-training-container',
 'registryId': '284415450706',
 'repositoryName': 'xgb-clf-training-container',
 'repositoryUri': '284415450706.dkr.ecr.eu-north-1.amazonaws.com/xgb-clf-training-container',
 'createdAt': datetime.datetime(2024, 9, 23, 17, 34, 6, 570000, tzinfo=tzlocal()),
 'imageTagMutability': 'MUTABLE',
 'imageScanningConfiguration': {'scanOnPush': False},
 'encryptionConfiguration': {'encryptionType': 'AES256'}}

In [5]:
lifecycle_policy = {
    "rules": [
        {
            "rulePriority": 1,
            "description": "Keep only one untagged image, expire all others",
            "selection": {
                "tagStatus": "any",
                "countType": "imageCountMoreThan",
                "countNumber": 1
            },
            "action": {
                "type": "expire"
            }
        }
    ]
}

ecr_manager.put_lifecycle_policy(processor_image_name, lifecycle_policy)
ecr_manager.put_lifecycle_policy(train_image_name, lifecycle_policy)

In [ ]:
tag = ':latest'

processor_image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, processor_image_name + tag)
print(f'Processor image name: {processor_image_uri}.')
!docker build -t $processor_image_uri ../preprocessor/docker
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
!docker push $processor_image_uri

In [ ]:
tag = ':latest'

train_image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, train_image_name + tag)
print(f'Train image name: {train_image_uri}.')
!docker build -t $train_image_uri ../containers/training
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
!docker push $train_image_uri

In [ ]:
from botocore.exceptions import ClientError
import logging

s3_client = aws_client_manager.get_client('s3')

try:
    s3_client.create_bucket(
        Bucket=bucket,
        CreateBucketConfiguration={'LocationConstraint': region}
    )
    logging.info(f"S3 bucket {bucket} created successfully.")
except s3_client.exceptions.BucketAlreadyOwnedByYou:
    logging.warning(f"S3 bucket {bucket} already exists and is owned by you.")
except ClientError as e:
    logging.error(f"Error creating S3 bucket: {e}")
    raise

In [ ]:
from sagemaker.session import Session

s3_location = f"s3://{bucket}/football"
sagemaker_session = Session()

df_local_path = str(os.environ['DATA_FILEPATH_X'])
y_local_path = str(os.environ['DATA_FILEPATH_Y'])

s3_client.upload_file(Filename=df_local_path, Bucket=bucket, Key=f"data/df.csv")
s3_client.upload_file(Filename=y_local_path, Bucket=bucket, Key=f"data/y.csv")